# <center>Welcome to my DSCI 230 Final Project!</center>
**We will be exploring some data supplied by a mountain biking company in Bentonville, AR. 
The company has customers who register before walking in, and customers who come without a reservation. We call these *registered* and *casual* customers. Here are some things we want to know:**
- If weather affects one group of customers more than another
- If there are thresholds in which a group may decide not to come (snow, rain, high temps, etc.)
- Explore a relationship between weather attributes and registered/casual customers

**We will use linear regression to do this. We'll start by loading in the libraries we need for this project:**

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib qt

**Now we will load our dataset and begin cleaning it. This includes finding where we have missing values, then filling those in. We also want to find any outliers in the data, and go ahead and remove those. Here, I create two helper functions to achieve this. Lastly, we remove columns we don't need, these include things like latitude, longitude, and elevation, which are values that don't change since all of this data was curated in the same location:**

In [2]:
data = pd.read_csv("ARBike.csv")
print("Shape of unfiltered data:", data.shape)
print("\nMissing value counts:\n",data.isna().sum())   # We can see there are just a few datapoints missing that we have to take care of

# This function will take a dataset and column name or list of names as input and fill NA values with the median
def fill_with_median(data, column_name):
    if isinstance(column_name, list):
        for col in column_name:
            impute_val = data[col].median()
            data[col] = data[col].fillna(impute_val)
    else:
        impute_val = data[column_name].median()
        data[column_name] = data[column_name].fillna(impute_val)

data.columns = data.columns.str.strip()  # Some columns had unnecessary extra whitespace characters
cols_with_NA = ["TMAX_10THC", "TMIN_10THC", "REGISTERED", "TTLCNT"]  
fill_with_median(data, cols_with_NA)
print("\nMissing value counts after cleaning:\n",data.isna().sum())   # Now we can see all the NA values are gone

# This function will take a dataset and a column name and filter outliers using IQR (interquartile range)
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    iqr = Q3 - Q1
    lower_bound = Q1 - 10 * iqr
    upper_bound = Q3 + 10 * iqr
    filtered_data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    return filtered_data

outlier_potentials = ["PRCP_MM", "TMAX_10THC", "TMIN_10THC", "AWND_MTRSPERSEC", "REGISTERED", "CASUAL", "TTLCNT"]
for col in outlier_potentials:
    data = remove_outliers_iqr(data, col)
    
# We will use indexing options to create a new dataset that has only the columns we need
# Columns like station, latitude, longitude, elevation, and name are not needed because 
# they're the same throughout the entire dataset
data = data[['DATE', 'PRCP_MM', 'TMAX_10THC', 'TMIN_10THC', 'AWND_MTRSPERSEC', 'CASUAL', 'REGISTERED', 'TTLCNT']]
data.head()

print("\nShape of filtered data:", data.shape)  # Our function detected 73 outliers, but now we know our data is clean

Shape of unfiltered data: (355, 13)

Missing value counts:
 STATION            0
DATE               0
LATITUDE           0
LONGITUDE          0
ELEVATION          0
NAME               0
PRCP_MM            0
TMAX_10THC         3
TMIN_10THC         1
AWND_MTRSPERSEC    0
CASUAL             0
REGISTERED         1
TTLCNT             1
dtype: int64

Missing value counts after cleaning:
 STATION            0
DATE               0
LATITUDE           0
LONGITUDE          0
ELEVATION          0
NAME               0
PRCP_MM            0
TMAX_10THC         0
TMIN_10THC         0
AWND_MTRSPERSEC    0
CASUAL             0
REGISTERED         0
TTLCNT             0
dtype: int64

Shape of filtered data: (313, 8)


**Next, we'll perform some operations on the data to familiarize ourselves with the dataset. In this cell, we're going to analyze a couple different things. These include:**
- Sorting the dataset by registered users to see what conditions brought the most registered customers
- Computing a boolean array to see which rows have a temperature of at least 200
- Using join operations to merge two subsets of the data together
- Using GroupBy methods to view total viewers by month and average users per weekday
- Slice the dataset to look at a certain month with DateTime
- Create a Period Index for quarterly analysis
- Find the distance between a date in the highest attended month vs the lowest attended month

In [3]:
# We can sort the dataset by registered users to see the what conditions brought the most registered users
highest_reg = data.sort_values(by='REGISTERED', ascending=False)
print(highest_reg.head())

# We can compute a boolean array to see which rows have a temp of at least 200
temp_bool_arr = data['TMAX_10THC'] >= 200
temp_bool_arr[temp_bool_arr == False].count()
print(temp_bool_arr.head())

# We can also demonstrate two different types of joins by creating two datasets and merging them together
df1 = data[['DATE','CASUAL', 'TMAX_10THC']].iloc[::2]
df2 = data[['DATE', 'REGISTERED']]

inner_join = pd.merge(df1, df2, on='DATE', how='left')
outer_join = pd.merge(df1, df2, on='DATE', how='right')
print("\n", inner_join.head())
print(outer_join.head())

# We can use 2 different groupby methods to view total number of users by month and average users per weekday
# First, convert 'DATE' column to datetime format
data['DATE'] = pd.to_datetime(data['DATE'])
# Total number of customers by month
monthly_users = data.groupby(data['DATE'].dt.month)[['REGISTERED', 'CASUAL']].sum()
print("\nTotal Number of Customers by Month:")
print(monthly_users)

# Average customers per weekday
weekday_users = data.groupby(data['DATE'].dt.weekday)[['REGISTERED', 'CASUAL']].mean()
print("\nAverage Customers per Weekday:")
print(weekday_users)

# We can slice the dataset to focus on a specific month with datetime
# We will look at August, since we saw from before it brought the highest registered customer attendance
august_data = data[data['DATE'].dt.month == 8]
print("\nCustomer Attendance in August:")
print(august_data[['DATE', 'PRCP_MM', 'TMAX_10THC', 'TMIN_10THC', 'CASUAL', 'REGISTERED']].head())

# We can create a period index for quarterly analysis
quarterly_index = pd.PeriodIndex(data['DATE'], freq='Q')
data['QUARTER'] = quarterly_index
print("\nQuarterly Analysis:")
print(data[['DATE', 'TMAX_10THC', 'TMIN_10THC', 'CASUAL', 'REGISTERED', 'QUARTER']].head())

# We can calculate the duration between two specific dates
# We will calculate the distance between a random date in the highest attended month and the lowest
date1 = pd.to_datetime('2021-08-21')
date2 = pd.to_datetime('2021-01-03')
duration = date1 - date2
print("\nDuration between", date1, "and", date2, ":", duration)

          DATE  PRCP_MM  TMAX_10THC  TMIN_10THC  AWND_MTRSPERSEC  CASUAL  \
234  8/29/2021        0       300.0       217.0               24    1281   
180   7/6/2021        0       306.0       189.0               16    1027   
241   9/5/2021        0       289.0       194.0               30     775   
179   7/5/2021        0       289.0       183.0               22     848   
272  10/7/2021        0       261.0       133.0               13     830   

     REGISTERED  TTLCNT  
234      4614.0  5895.0  
180      4488.0  5515.0  
241      4429.0  5204.0  
179      4377.0  5225.0  
272      4372.0  5202.0  
1    False
2    False
3    False
4    False
7    False
Name: TMAX_10THC, dtype: bool

         DATE  CASUAL  TMAX_10THC  REGISTERED
0   1/2/2021     131        11.0       670.0
1   1/4/2021     108       122.0      1454.0
2   1/8/2021      68        28.0       891.0
3  1/10/2021      41        17.0      1280.0
4  1/12/2021      25        89.0      1137.0
       DATE  CASUAL  TMAX_10TH

**Next up, we'll create some visualizations. This includes line plots, vertical and horizontal plots, scatter plots, and box plots. This will help us draw conclusions and make inferences on our data. The first one we'll make is a line plot that will look at the number of casuals and registered customers over the year:**

In [4]:
# We will start by splitting our data into 2 sets: casual and registered
casual = data[['DATE', 'PRCP_MM', 'TMAX_10THC', 'TMIN_10THC', 'AWND_MTRSPERSEC', 'CASUAL']]
registered = data[['DATE', 'PRCP_MM', 'TMAX_10THC', 'TMIN_10THC', 'AWND_MTRSPERSEC', 'REGISTERED']]

# Now we will create a line plot with point markers for registered customers
plt.figure(figsize=(10, 6))

# Plot registered customers
plt.plot(registered['DATE'], registered['REGISTERED'], marker='o', linestyle='-', color='green', label='Registered Customers')
# Plot casual customers
plt.plot(casual['DATE'], casual['CASUAL'], marker='o', linestyle='-', color='darkblue', label='Casual Customers')
# Add details like labels and a legend
plt.xlabel('Date')
plt.ylabel('Number of Customers')
plt.title('Registered and Casual Customer Counts over the Year')
plt.xticks(rotation=45)
plt.xticks(registered['DATE'][::15])
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**We can see from this plot that the company gets a significantly larger amount of registered customers than casual customers. This is pretty expected for a destination regarded as having 'world-class mountain biking trails'. We can also see that their peak number of customers comes in the summer months. This is also pretty expected due to warmer weather and more people traveling/on vacation. So from this, we can pretty safely infer that the time of year doesn't trend to more registered customers visiting compared to casual, or vice versa.**

**Next, we'll look at two different bar plots. One will view the number of customers by wind speed, and the other will view the number of customers by the amount of precipitation:**

In [5]:
# Vertical Bar Plot: Number of customers by wind speed
wind_users = data.groupby('AWND_MTRSPERSEC')[['REGISTERED', 'CASUAL']].sum()
wind_users.plot(kind='bar', figsize=(10, 6), color=['red', 'blue'])
plt.title('Number of Customers in Relation to Wind Speeds')
plt.xlabel('Wind Speed (meters per second)')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)
plt.legend(['Registered', 'Casual'])
plt.grid()
plt.tight_layout()
plt.show()

# Horizontal Bar Plot: Number of customers by amount of rain/snow
weather_users = data.groupby('PRCP_MM')[['REGISTERED', 'CASUAL']].sum()
weather_users.plot(kind='barh', figsize=(10, 6), color=['skyblue', 'purple'])
plt.title('Number of Customers in Relation to Amount of Precipitation')
plt.xlabel('Number of Customers')
plt.ylabel('Milimeters of Rain/Snow')
plt.legend(['Registered', 'Casual'])
plt.grid()
plt.show()

**There are some obvious observations for these plots as well. We can see that attendance drops off a cliff when there is essentially any rain. This makes sense because let's be honest, who wants to hike or go on bike trails in the rain? The other plot, however, shows a huge spike in attendance at a wind speed of 30 meters per second. This is likely either a coincidence or the data just has a lot of samples with a wind speed of 30 m/s. Outside of that, everything looks pretty standard. To me, it doesn't look like wind speed has a huge correlation with attendance, I think it's just moreso that we have a lot more data samples in the 15-40 m/s range than we do anything more or less than that. From this, we can infer that neither wind speeds or precipitation amounts will lead to more/less attendance from one group of customers compared to the other.**

**Next we'll view a scatter plot comparing the high temperature for a given day and the number of total customers that came to the mountain biking company that day. We will also fit a regression line on this plot.**

In [6]:
# Scatter Plot with regression lines for REGISTERED customers
sns.regplot(x='TMAX_10THC', y='REGISTERED', data=data, scatter_kws={'color': 'blue'}, line_kws={'color': 'blue'})
# Scatter Plot with regression lines for CASUAL customers
sns.regplot(x='TMAX_10THC', y='CASUAL', data=data, scatter_kws={'color': 'orange'}, line_kws={'color': 'orange'})

plt.title('Number of Total Customers in Relation to Increasing Temperatures')
plt.xlabel('Maximum Temperature')
plt.ylabel('Number of Customers')

# The legend wouldn't work right, so I had to jump through some hoops for it
legend_handles = [plt.Line2D([0], [0], color='blue', marker='o', linestyle='None', label='REGISTERED'),
                  plt.Line2D([0], [0], color='orange', marker='s', linestyle='None', label='CASUAL')]
plt.legend(handles=legend_handles)
plt.show()

**We can see from the plot that there's a decent correlation between the number of customers and the temperature. This supports our findings in our previous plot that showed higher attendance during summer months. There doesn't seem to be anything out of the ordinary here either. Both groups seem to go biking more in warmer weather, regardless of if it's a *registered* or *casual* customer.**

**Now we're gonna look at a box plot. This will categorize a continuous variable. For this example, we're going to split the dates into the four seasons, and then view customer attendance in relation to the season.**

In [7]:
# We first extract the month from the 'DATE' column
data['MONTH'] = data['DATE'].dt.month

# Then we define a helper function to group by season
def get_season(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    elif month in [9,10,11]:
        return 'Fall'
    else:
        raise Exception('Error, month data not within required range')

# Now we apply the function to create a 'SEASON' column
data['SEASON'] = data['MONTH'].apply(get_season)

# Finally, we create the box plot
plt.figure(figsize=(10,6))
sns.boxplot(x='SEASON', y='TTLCNT', data=data)
plt.title('Number of Total Customers by Season')
plt.xlabel('Season')
plt.ylabel('Number of Total Customers')
plt.show()

**There's some interesting observations to be made here. First, I noticed the median line in the winter box isn't close to the middle of the box, meaning we've probably got some skewed data there. We can also see that the box plot for the summer box is pretty small, meaning the data is clustered around the median. Looking at the big picture, summer and fall months attract the most customers, with spring not far behind, while winter is firmly in last. It's also worth noting that we've got some outliers in the summer and fall boxes.**

**Next, we're going to try and create a model that will look at weather features for each day, and try to predict if someone will go biking that day. To start, we'll make a scatter plot with a linear regression line showing the difference between *registered* and *casual* customers' attendance in regards to the low temps, since it's the only weather attribute of the four in the dataset that we haven't looked at yet:**

In [8]:
# Scatter Plot with regression lines for REGISTERED customers
sns.regplot(x='TMIN_10THC', y='REGISTERED', data=data, scatter_kws={'color': 'darkblue'}, line_kws={'color': 'darkblue'})
# Scatter Plot with regression lines for CASUAL customers
sns.regplot(x='TMIN_10THC', y='CASUAL', data=data, scatter_kws={'color': 'lightblue'}, line_kws={'color': 'lightblue'})

plt.title('Number of Total Customers in Relation to Low Temperatures')
plt.xlabel('Minimum Temperature')
plt.ylabel('Number of Customers')
legend_handles = [plt.Line2D([0], [0], color='darkblue', marker='o', linestyle='None', label='REGISTERED'),
                  plt.Line2D([0], [0], color='lightblue', marker='s', linestyle='None', label='CASUAL')]
plt.legend(handles=legend_handles)

# Reverse x-axis to go from high temps to low temps instead of low to high
plt.xlim(plt.xlim()[::-1])
plt.show()

**We can clearly see just like in the scatterplot that showed higher attendance for warm days, we see a big drop in attendance when the temperature is cool. Next, we're gonna work on developing the model. We will first put our 'predictors' in an array:**

In [9]:
# 'predictors' in X array
X = data[['PRCP_MM', 'TMAX_10THC', 'TMIN_10THC', 'AWND_MTRSPERSEC']]

**Now, here's the tricky part. We don't have access to data saying whether a customer came or did not come on any given day. So we will have to curate this data ourselves and get a little creative with it.**

In [10]:
# First we calculate mean and standard deviation of each predictor
mean_prcp = data['PRCP_MM'].mean()
std_prcp = data['PRCP_MM'].std()
mean_maxtemp = data['TMAX_10THC'].mean()
std_maxtemp = data['TMAX_10THC'].std()
mean_mintemp = data['TMIN_10THC'].mean()
std_mintemp = data['TMIN_10THC'].std()
mean_wind_speed = data['AWND_MTRSPERSEC'].mean()
std_wind_speed = data['AWND_MTRSPERSEC'].std()

# Now define a function to determine if weather attribute is close enough to the mean
def is_close_enough(value, mean_attribute, std_attribute):
    # Threshold sets data to where it has to be within one standard deviation
    threshold = 1 * std_attribute
    return abs(value - mean_attribute) <= threshold

def will_bike(row):
    # Now we use that function to generate boolean arrays for each attribute
    prcp_close = is_close_enough(row['PRCP_MM'], mean_prcp, std_prcp)
    maxtemp_close = is_close_enough(row['TMAX_10THC'], mean_maxtemp, std_maxtemp)
    mintemp_close = is_close_enough(row['TMIN_10THC'], mean_mintemp, std_mintemp)
    wind_close = is_close_enough(row['AWND_MTRSPERSEC'], mean_wind_speed, std_wind_speed)
    
    if prcp_close and maxtemp_close and mintemp_close and wind_close:
        return 1
    else:
        return 0

# Finally, apply the function and create a new column    
data['WILL_BIKE'] = data.apply(will_bike, axis=1)

**The method I used to create our 'WILL_BIKE' column is fairly simple. We could have gotten *way* more complex with it, but I figured this was all that was necessary for this project. Now that we have our 'y' column, we can continue with developing the model. What we'll do next is split our data into train and test sets, then use logistic regression to fit the model, and ultimately predict whether or not a potential customer will bike on a given day.**

In [11]:
# Now use our new column as our 'y'
y = data['WILL_BIKE']

# We use train_test_split function to split data into training and testing sets (80% train, 20% test)
# This is from sk_learn library, handy for when you don't have separate train and test datasets already
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now instantiate the logistic regression model
model = LogisticRegression(C=10)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the testing data
y_pred = model.predict(X_test)

# Score the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Display summary statistics
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
y_pred = model.predict(X_test)
print("R-squared:", r2_score(y_test, y_pred))

Accuracy: 0.746031746031746
Coefficients: [[-0.2544129   0.00728958 -0.00686684  0.00876838]]
Intercept: [-1.6636494]
R-squared: -0.24444444444444513


**And finally, we can see that our model graded out with a 74.6% accuracy. There are also other summary statistics displayed such as coefficients, y-intercept value, and the R-squared value. This means that our model predicts whether or not a potential customer will bike that day with 74.6% accuracy. Not great, but not terrible either. The bike store owner could apply this model to maybe adjust prices or adjust schedules on a given work week, depending on what the weather looks like.**

**That concludes my data analysis project. Please refer to the word doc for a write-up of my findings. This project was pretty fun to work with and I hope you enjoyed learning along with me!**